In [99]:
# ==============================================================================
# BAGIAN 1 (VERSI FINAL): BACA & BERSIHKAN FILE "KOTOR" (ADVANCED)
# ==============================================================================
import pandas as pd
import numpy as np
from google.colab import drive
import csv # Kita gunakan modul CSV bawaan Python

print("Menghubungkan ke Google Drive...")
drive.mount('/content/drive')
print("Google Drive Terhubung!")

# --- Path Anda sudah benar ---
PATH_DATA_ANDA = "/content/drive/MyDrive/For Proyek/Proyek 1 ATM/data_atm_downtime_FINAL.csv"
print(f"Membaca file dari: {PATH_DATA_ANDA}")

# Kita siapkan "wadah" kosong untuk data bersih kita
data_bersih = []
header = []
jumlah_baris_rusak = 0
baris_pertama = True
expected_columns = 18 # Kita tahu seharusnya ada 18 kolom

# Kita buka file .csv sebagai file teks biasa
with open(PATH_DATA_ANDA, mode='r', encoding='utf-8') as file:
    # Kita gunakan csv.reader untuk membaca baris demi baris
    csv_reader = csv.reader(file)

    for row in csv_reader:
        # --- Safety Check 1: Lewati baris kosong ---
        if not row:
            continue

        # 1. Ambil header (baris pertama)
        if baris_pertama:
            header = row
            expected_columns = len(header)
            print(f"Header terdeteksi dengan {expected_columns} kolom.")
            baris_pertama = False
            continue  # Lanjut ke baris berikutnya

        # --- Safety Check 2: Pastikan baris punya data (minimal ID) ---
        if len(row) < 2:
            print(f"Melewatkan baris aneh (terlalu pendek): {row}")
            continue

        # --- PERBAIKAN KHUSUS (FOTO 1: Baris "Music") ---
        if row[1] == '1737': # Cek berdasarkan ID unik
            print("Memperbaiki baris 'Music' (ID 1737)...")
            # Ini data yang "bergeser": [..., 'Padalarang', '14-Nov-25', 'Music', '19204495', 'Telkom', ...]
            # Kita "buang" 'Music' (indeks 4)
            fixed_row = row[:4] + row[5:]

            # Sekarang jumlah kolomnya 17, kita tambahkan 1 kolom kosong di akhir
            # agar pas 18 (karena kolom 'Penanggung jawab' memang kosong di data dummy)
            if len(fixed_row) == expected_columns - 1:
                fixed_row.append('')

            if len(fixed_row) == expected_columns:
                 data_bersih.append(fixed_row)
            continue # Lanjut ke baris berikutnya

        # 2. Cek baris yang "BERSIH" (jumlah kolom = 18)
        if len(row) == expected_columns:
            data_bersih.append(row)

        # 3. SOLUSI UTAMA: Perbaiki baris "RUSAK" (kolom > 18 karena koma nyasar)
        elif len(row) > expected_columns:
            jumlah_baris_rusak += 1

            # Asumsi koma nyasar ada di 'Penyebab' (indeks 12)
            # Cth: 'Listrik', ' ruko mati'

            # Hitung berapa banyak kolom ekstra
            extra_columns = len(row) - expected_columns

            # Gabungkan kolom 'Penyebab' yang terpecah
            fixed_penyebab = ", ".join(row[12 : 13 + extra_columns])

            # Buat ulang baris (row) yang sudah diperbaiki
            fixed_row = row[:12] + [fixed_penyebab] + row[13 + extra_columns:]

            # Pastikan lagi jumlah kolomnya sudah 18
            if len(fixed_row) == expected_columns:
                data_bersih.append(fixed_row)
            else:
                # Jika masih gagal, mungkin ada 2 koma nyasar
                # Coba lagi dengan asumsi 2 koma
                extra_columns = len(row) - expected_columns
                fixed_penyebab = ", ".join(row[12 : 13 + extra_columns])
                fixed_row = row[:12] + [fixed_penyebab] + row[13 + extra_columns:]

                if len(fixed_row) == expected_columns:
                    data_bersih.append(fixed_row)
                else:
                     print(f"Gagal memperbaiki baris (masih {len(fixed_row)} kolom): {row}")

        # 4. (Opsional) Lewati baris yang terlalu rusak/aneh
        else:
            print(f"Melewatkan baris aneh (kolom < {expected_columns}): {row}")

# 5. Konversi data bersih kita menjadi DataFrame Pandas
df = pd.DataFrame(data_bersih, columns=header)

print("\n--- HASIL PEMBACAAN DATA ---")
print(f"Total baris data yang berhasil dibaca & dibersihkan: {len(df)}")
print(f"Total baris yang 'rusak' (koma nyasar) dan berhasil diperbaiki: {jumlah_baris_rusak}")
print(f"Data asli (setelah diperbaiki) berhasil dimuat. {df.shape}")


Menghubungkan ke Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive Terhubung!
Membaca file dari: /content/drive/MyDrive/For Proyek/Proyek 1 ATM/data_atm_downtime_FINAL.csv
Header terdeteksi dengan 18 kolom.
Memperbaiki baris 'Music' (ID 1737)...

--- HASIL PEMBACAAN DATA ---
Total baris data yang berhasil dibaca & dibersihkan: 100
Total baris yang 'rusak' (koma nyasar) dan berhasil diperbaiki: 21
Data asli (setelah diperbaiki) berhasil dimuat. (100, 18)


In [100]:
# ==============================================================================
# BAGIAN 2: MEMBUAT "SHEET LIST" (Data Dictionaries)
# ==============================================================================
# Ini adalah terjemahan dari "Sheet List" Anda di Excel.
# Kita membuat 'kamus' di Python agar kode lebih rapi.

# Opsi isian dari Sheet List Kolom D (Real Problem)
list_d = {
    "D2": "Dispenser", "D3": "Kaset", "D4": "Restart Mesin/Modem", "D5": "Self Recovery",
    "D6": "Uang Tersangkut", "D7": "Grounding", "D8": "UPS", "D9": "Jaringan", "D10": "MCB",
    "D11": "Installasi software/ Reinstall software", "D12": "Config ATM", "D13": "Kartu Tersangkut",
    "D14": "Riset /Restart/cleaning", "D15": "Token Listrik", "D16": "Pengisian Uang",
    "D17": "Pemadaman Listrik PLN", "D18": "Pemadaman Listrik Gedung", "D19": "Replace Part",
    "D20": "Pending Part", "D21": "Real estate", "D22": "Environment", "D23": "Tidak Ada Akses",
    "D24": "Config komponen", "D25": "On-Progress", "D26": "Vandalisme", "D27": "Card Reader"
    # Dll. sesuai list lengkap Anda
}

# Opsi isian dari Sheet List Kolom C (Kategori)
list_c = {
    "C2": "Hardware", "C3": "Software", "C4": "Self recovery", "C9": "Kelistrikan",
    "C10": "Link", "C12": "Force Majour", "C13": "On-Progress"
    # Dll. sesuai list lengkap Anda
}

# Opsi isian dari Sheet List Kolom B (Responsibility)
list_b = {
    "B2": "Vendor ATM SIMPON", "B4": "Vendor ATM ADABUS", "B5": "Vendor ATM Kejar", "B15": "KEMON"
    # Dll. sesuai list lengkap Anda
}

print("Data 'List' (Kamus) berhasil dibuat.")


Data 'List' (Kamus) berhasil dibuat.


In [101]:
# ==============================================================================
# BAGIAN 3: "RULE-BASED ENGINE" - PIPA 1 (Kolom M -> Kolom N)
# ==============================================================================
# Versi ini menggunakan prioritas awal (yang "salah"),
# TAPI sudah diperbarui dengan KAMUS KATA KUNCI TERLENGKAP.

print("Memulai Pipa 1: Menentukan 'Real problem' (Kolom N)...")

# Buat kolom input yang bersih (kolom 'Penyebab' / M2)
df['Penyebab_clean'] = df['Penyebab'].astype(str).str.lower().fillna('')

# 1. Definisikan semua KONDISI (IFERROR(SEARCH...))
# --- KITA GUNAKAN URUTAN PRIORITAS AWAL ANDA ---
conditions_real_problem = [
    # (DIPERBAIKI) Aturan D2: Tambahkan 'kertas' dan 'struk'
    df['Penyebab_clean'].str.contains('dispenser|cash|kertas|struk', na=False),

    df['Penyebab_clean'].str.contains('kaset|casete|cassete', na=False),
    df['Penyebab_clean'].str.contains('restart|restart mesin|rest all device|cek mesin|modem|reset', na=False),
    df['Penyebab_clean'].str.contains('updown|up/down|gasper|jarkom|normal|mesin online|sudah dalam keadaan online|tim sampai lokasi|tidak ada info problem|mesin sudah terpantau online|mesin sudah online|mesin atm down online sendiri|dilokasi online,transaksi ok|tidak ada info problem kepada kami|tidak ada info apapun|team sampai lokasi mesin sudah online kembali|di lokasi transaksi ok|sel', na=False),

    # (DIPERBAIKI) Aturan D6: Tambahkan 'uang tidak terdeteksi', 'uang tidak terhitung'
    df['Penyebab_clean'].str.contains('uang tersangkut|temuan uang nyangkut|uang|uang tidak terdeteksi|uang tidak terhitung', na=False),

    df['Penyebab_clean'].str.contains('ups', na=False),

    # Aturan 'jaringan' tetap di sini (prioritas lebih tinggi dari 'open tiket')
    df['Penyebab_clean'].str.contains('jaringan|jarinagan|server|di lokasi gangguan komunikas|gangguan komunikasi', na=False),

    df['Penyebab_clean'].str.contains('install|software|update bios|ej', na=False),

    # (DIPERBAIKI) Aturan D13: Tambahkan 'kartu tidak bisa keluar'
    df['Penyebab_clean'].str.contains('kartu tersangkut|kartu tidak bisa keluar|kartu nyangkut', na=False),

    df['Penyebab_clean'].str.contains('riset|pm|sensor/clean vacum|action clean vacum|cek receipt , test printer, atm ok|cleaning', na=False),
    df['Penyebab_clean'].str.contains('token', na=False),
    df['Penyebab_clean'].str.contains('pengisian|replanish|replenish|rpl', na=False),

    # (DIPERBAIKI FINAL) Aturan D17: Tambahkan 'listrik mati', 'mati listrik', 'mati lampu'
    df['Penyebab_clean'].str.contains('pln|pemadaman|listrik padam|problem kelistrikan|padam|listrik mati|mati listrik|mati lampu', na=False),

    df['Penyebab_clean'].str.contains('gedung|konsleting listrik|mcb', na=False),
    df['Penyebab_clean'].str.contains('penggantian|pasang|replace', na=False),
    df['Penyebab_clean'].str.contains('pending|menunggu', na=False),
    df['Penyebab_clean'].str.contains('lokasi tutup|lokasi non 24 jam|oprasional|toko tutup|akses', na=False),

    # Aturan 'open tiket' tetap di sini (prioritas rendah)
    df['Penyebab_clean'].str.contains('teknisi|tiket|slm|open|vendor|tiket jarkom terkait offline|ot', na=False),

    # (DIPERBAIKI FINAL) Aturan D27: Tambahkan 'kartu tidak bisa dibaca', 'kartu tidak terdeteksi', 'kartu tidak terbaca'
    df['Penyebab_clean'].str.contains('counter|crd|cd|card|card reader|test card|reset card|hapus problem|reset card reader|cr|kartu tidak bisa dibaca|kartu tidak terdeteksi|kartu tidak terbaca', na=False)
]

# 2. Definisikan semua PILIHAN (List!$D$n)
# --- URUTAN PILIHAN SESUAI DENGAN KONDISI DI ATAS ---
choices_real_problem = [
    list_d['D2'], list_d['D3'], list_d['D4'], list_d['D5'], list_d['D6'],
    list_d['D8'],

    list_d['D9'], # Pilihan "Jaringan"

    list_d['D11'],

    list_d['D13'], # Pilihan "Kartu Tersangkut"

    list_d['D14'], list_d['D15'], list_d['D16'],

    list_d['D17'], # Pilihan "Listrik PLN"

    list_d['D18'], list_d['D19'],
    list_d['D20'], list_d['D23'],

    list_d['D25'], # Pilihan "On-Progress"

    list_d['D27']  # Pilihan "Card Reader"
]

# 3. Jalankan "Mesin"
df['N_Real_Problem'] = np.select(conditions_real_problem, choices_real_problem, default='')

print("Pipa 1 Selesai. Kolom 'N_Real_Problem' telah dibuat (Versi LOGIKA TERLENGKAP).")


Memulai Pipa 1: Menentukan 'Real problem' (Kolom N)...
Pipa 1 Selesai. Kolom 'N_Real_Problem' telah dibuat (Versi LOGIKA TERLENGKAP).


In [102]:
# ==============================================================================
# BAGIAN 4: "RULE-BASED ENGINE" - PIPA 2 (Kolom N -> Kolom P)
# ==============================================================================
# Menerjemahkan Formula 2 Anda (Kategori).
# Input: 'N_Real_Problem' (Kolom N)
# Output: 'P_Kategori' (Kolom P)

print("\nMemulai Pipa 2: Menentukan 'Kategori' (Kolom P)...")

# 1. Definisikan KONDISI (IF(OR(N3=List!$D$2...)))
conditions_kategori = [
    (df['N_Real_Problem'] == list_d['D2']) | (df['N_Real_Problem'] == list_d['D3']) | (df['N_Real_Problem'] == list_d['D4']) | (df['N_Real_Problem'] == list_d['D6']) | (df['N_Real_Problem'] == list_d['D13']) | (df['N_Real_Problem'] == list_d['D14']) | (df['N_Real_Problem'] == list_d['D16']) | (df['N_Real_Problem'] == list_d['D27']) | (df['N_Real_Problem'] == list_d['D19']),
    (df['N_Real_Problem'] == list_d['D11']) | (df['N_Real_Problem'] == list_d['D12']),
    (df['N_Real_Problem'] == list_d['D8']) | (df['N_Real_Problem'] == list_d['D15']) | (df['N_Real_Problem'] == list_d['D17']) | (df['N_Real_Problem'] == list_d['D18']),
    (df['N_Real_Problem'] == list_d['D20']) | (df['N_Real_Problem'] == list_d['D25']),
    (df['N_Real_Problem'] == list_d['D23']),
    (df['N_Real_Problem'] == list_d['D9']),
    (df['N_Real_Problem'] == list_d['D5'])
]

# 2. Definisikan PILIHAN (List!$C$n)
choices_kategori = [
    list_c['C2'], list_c['C3'], list_c['C9'], list_c['C13'], list_c['C12'],
    list_c['C10'], list_c['C4']
]

# 3. Jalankan "Mesin"
df['P_Kategori'] = np.select(conditions_kategori, choices_kategori, default='')

print("Pipa 2 Selesai. Kolom 'P_Kategori' telah dibuat.")



Memulai Pipa 2: Menentukan 'Kategori' (Kolom P)...
Pipa 2 Selesai. Kolom 'P_Kategori' telah dibuat.


In [103]:
# ==============================================================================
# BAGIAN 5: "RULE-BASED ENGINE" - PIPA 3 (Kolom P + R -> Kolom Q)
# ==============================================================================
# Menerjemahkan Formula 3 Anda (Responsibility).
# Input 1: 'P_Kategori' (Kolom P)
# Input 2: 'Penanggung jawab' (Kolom R di data Anda, S3 di formula Anda)
# Output: 'Q_Responsibility' (Kolom Q)

print("\nMemulai Pipa 3: Menentukan 'Responsibility' (Kolom Q)...")

# --- PERBAIKAN DI SINI ---
# Kita tambahkan .str.upper()
df['R_PJ_Uptime_clean'] = df['Penanggung jawab'].astype(str).str.strip().str.upper()
# -------------------------

# 1. Definisikan KONDISI (IF(AND(OR...)))
#    Kita buat 1 kondisi besar untuk 'kategori'
kondisi_kategori_umum = (df['P_Kategori'] == list_c['C4']) | \
                        (df['P_Kategori'] == list_c['C12']) | \
                        (df['P_Kategori'] == list_c['C2']) | \
                        (df['P_Kategori'] == list_c['C9']) | \
                        (df['P_Kategori'] == list_c['C10']) | \
                        (df['P_Kategori'] == list_c['C3'])
                        # (Saya hanya memasukkan yang ada di formula Anda)

# Gabungkan dengan kondisi 'Penanggung jawab'
conditions_responsibility = [
    kondisi_kategori_umum & (df['R_PJ_Uptime_clean'] == "ADABUS"),
    kondisi_kategori_umum & (df['R_PJ_Uptime_clean'] == "SIMPON"),
    kondisi_kategori_umum & (df['R_PJ_Uptime_clean'] == "KEJAR"),
    kondisi_kategori_umum & (df['R_PJ_Uptime_clean'] == "KEMON")
]

# 2. Definisikan PILIHAN (List!$B$n)
choices_responsibility = [
    list_b['B4'], list_b['B2'], list_b['B5'], list_b['B15']
]

# 3. Jalankan "Mesin"
df['Q_Responsibility'] = np.select(conditions_responsibility, choices_responsibility, default='')

print("Pipa 3 Selesai. Kolom 'Q_Responsibility' telah dibuat.")



Memulai Pipa 3: Menentukan 'Responsibility' (Kolom Q)...
Pipa 3 Selesai. Kolom 'Q_Responsibility' telah dibuat.


In [104]:
# ==============================================================================
# BAGIAN 6: VERIFIKASI HASIL AKHIR (SESUAI URUTAN ASLI)
# ==============================================================================

# --- Kode untuk memaksa tampilan mendatar ---
pd.set_option('display.max_rows', None)     # Tampilkan semua baris
pd.set_option('display.max_columns', None)  # Tampilkan semua kolom
pd.set_option('display.width', 2000)        # Atur lebar tampilan super lebar
# ---------------------------------------------

print("\n\n--- 🏁 HASIL AKHIR PIPELINE OTOMATISASI (URUTAN ASLI) 🏁 ---")

# 1. Kita "salin" hasil dari kolom Python baru kita ke kolom asli yang kosong
df['Real problem'] = df['N_Real_Problem']
df['Kategori'] = df['P_Kategori']
df['Responsibilty'] = df['Q_Responsibility']


# 2. Tentukan urutan SEMUA kolom asli persis seperti data dummy
kolom_urutan_asli = [
    'No',
    'Id',
    'Lokasi',
    'Tanggal',
    'No tiket',
    'Komunikasi',
    'Jam operasional',
    'City',
    'Provinsi',
    'Durasi',
    'Start time',
    'End time',
    'Penyebab',          # Input Mentah (M)
    'Real problem',      # Output Pipa 1 (N) - SUDAH DIISI
    'Problem',
    'Kategori',          # Output Pipa 2 (P) - SUDAH DIISI
    'Responsibilty',     # Output Pipa 3 (Q) - SUDAH DIISI
    'Penanggung jawab'   # Input Tarikan Uptime (R/S)
]

# 3. Tampilkan 20 baris pertama dari SEMUA kolom dalam urutan asli
print(df[kolom_urutan_asli].head(20))

print("\n--- PROYEK 1 (MIGRASI LOGIKA) SELESAAI ---")




--- 🏁 HASIL AKHIR PIPELINE OTOMATISASI (URUTAN ASLI) 🏁 ---
    No    Id            Lokasi    Tanggal  No tiket Komunikasi Jam operasional        City     Provinsi Durasi           Start time             End time                            Penyebab           Real problem       Problem       Kategori      Responsibilty Penanggung jawab
0    1  1646  Bintaro Sektor 9  12-Nov-25  19204404    Sanatel   00:00 - 23:59     Jakarta  DKI Jakarta  01:30  11/12/2025 02:00 AM  11/12/2025 03:30 AM                mesin online sendiri          Self Recovery   Atm offline  Self recovery  Vendor ATM SIMPON           SIMPON
1    2  1647     Plaza Senayan  12-Nov-25  19204405     Telkom   00:00 - 23:59     Jakarta  DKI Jakarta  00:45  11/12/2025 02:15 AM  11/12/2025 03:00 AM                restart mesin atm ok    Restart Mesin/Modem   Atm offline       Hardware  Vendor ATM ADABUS           ADABUS
2    3  1648     Ciputra World  12-Nov-25  19204406    Indosat   08:00 - 20:00    Surabaya   Jawa Timur  05: